In [1]:
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
ENV["COLUMNS"] = 200;

In [2]:
using Dates
using CSV
using DataFrames

In [3]:
using JuMP
using Gurobi
using LinearAlgebra
using MathOptInterface

In [4]:
using BedsData
using ForecastData
using GeographicData

In [5]:
function patient_allocation(
        beds::Array{Float32,1},
        initial_patients::Array{Float32,1},
        admitted_patients::Array{Float32,2},
        initial_patients_discharged::Array{Float32,2},
        adj_matrix::BitArray{2};
        objective::Symbol=:overflow,
        hospitalized_days::Int=8,
        send_new_only::Bool=true,
        send_wait_period::Int=0,
        min_send_amt::Real=0,
        smoothness_penalty::Real=0,
        setup_cost::Real=0,
        sent_penalty::Real=0,
        verbose::Bool=false,
)
    N, T = size(admitted_patients)
    @assert(size(initial_patients, 1) == N)
    @assert(size(beds, 1) == N)
    @assert(size(adj_matrix) == (N,N))
    @assert(size(initial_patients_discharged) == (N, hospitalized_days))
    @assert(objective == :overflow)

    model = Model(Gurobi.Optimizer)
    if !verbose set_silent(model) end

    @variable(model, sent[1:N,1:N,1:T])
    @variable(model, obj_dummy[1:N,1:T] >= 0)

    if min_send_amt <= 0
        @constraint(model, sent .>= 0)
    else
        @constraint(model, [i=1:N,j=1:N,t=1:T], sent[i,j,t] in MOI.Semicontinuous(Float64(min_send_amt), Inf))
    end

    objective = @expression(model, sum(obj_dummy))
    if sent_penalty > 0
        add_to_expression!(objective, sent_penalty*sum(sent))
    end
    if smoothness_penalty > 0
        @variable(model, smoothness_dummy[i=1:N,j=1:N,t=1:T-1] >= 0)
        @constraint(model, [t=1:T-1],  (sent[:,:,t] - sent[:,:,t+1]) .<= smoothness_dummy[:,:,t])
        @constraint(model, [t=1:T-1], -(sent[:,:,t] - sent[:,:,t+1]) .<= smoothness_dummy[:,:,t])

        add_to_expression!(objective, smoothness_penalty * sum(smoothness_dummy))
        add_to_expression!(objective, smoothness_penalty * sum(sent[:,:,1]))
    end
    if setup_cost > 0
        @variable(model, setup_dummy[i=1:N,j=i+1:N], Bin)
        @constraint(model, [i=1:N,j=i+1:N], [1-setup_dummy[i,j], sum(sent[i,j,:])+sum(sent[j,i,:])] in MOI.SOS1([1.0, 1.0]))
        add_to_expression!(objective, setup_cost*sum(setup_dummy))
    end
    @objective(model, Min, objective)

    if send_new_only
        @constraint(model, [t=1:T],
            sum(sent[:,:,t], dims=2) .<= admitted_patients[:,t]
        )
    else
        @constraint(model, [t=1:T],
            sum(sent[:,:,t], dims=2) .<=
                initial_patients - sum(initial_patients_discharged[:,1:min(t,hospitalized_days)], dims=2)
                .+ sum(admitted_patients[:,max(1,t-hospitalized_days):t], dims=2)
                .- sum(sent[:,:,1:t-1], dims=[2,3])
                .+ sum(sent[:,:,max(1,t-hospitalized_days):t-1], dims=[1,3])
        )
    end

    for i = 1:N
        for j = 1:N
            if ~adj_matrix[i,j]
                @constraint(model, sum(sent[i,j,:]) .== 0)
            end
        end
    end

    if send_wait_period > 0
        @constraint(model, [i=1:N,t=1:T-1],
            [sum(sent[:,i,t]), sum(sent[i,:,t:min(t+send_wait_period,T)])] in MOI.SOS1([1.0, 1.0])
        )
    end

    flip_sign = (objective == :shortage) ? 1 : -1
    z1, z2 = (objective == :shortage) ? (0, -1) : (-1, 0)
    @constraint(model, [i=1:N,t=1:T],
        obj_dummy[i,t] >= flip_sign * (
            beds[i] - (
                initial_patients[i] - sum(initial_patients_discharged[i,1:min(t,hospitalized_days)])
                + sum(admitted_patients[i,max(1,t-hospitalized_days):t])
                - sum(sent[i,:,1:t+z1])
                + sum(sent[:,i,max(1,t-hospitalized_days):t+z2])
            )
        )
    )

    optimize!(model)
    return model
end;

In [6]:
ne_states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 8;

In [7]:
beds = compute_beds(ne_states, pct_beds_available=pct_beds_available)
forecast_start, _ = compute_ihme_forecast_net(start_date, end_date, ne_states)
forecast_admitted = compute_ihme_forecast_admitted(start_date, end_date, ne_states)
forecast_start_discharged = compute_ihme_forecast_admitted(start_date - Dates.Day(hospitalized_days-1), start_date, ne_states)
adj = compute_adjacencies(ne_states, hrs=travel_threshold_hours);

In [8]:
model = patient_allocation(
    beds,
    forecast_start,
    forecast_admitted,
    forecast_start_discharged,
    adj,
    hospitalized_days=hospitalized_days,
    send_new_only=true,
    send_wait_period=5,
    min_send_amt=10,
    smoothness_penalty=0.01,
    setup_cost=0.01,
    sent_penalty=0,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 8765 rows, 8547 columns and 136954 nonzeros
Model fingerprint: 0xe41c079c
Model has 385 SOS constraints
Variable types: 4741 continuous, 55 integer (55 binary)
Semi-Variable types: 3751 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [2e-02, 1e+04]
Presolve removed 4483 rows and 4418 columns
Presolve time: 0.25s
Presolved: 7576 rows, 5776 columns, 69027 nonzeros
Presolved model has 270 SOS constraint(s)
Variable types: 4037 continuous, 1739 integer (1739 binary)
Found heuristic solution: objective 175889.72417

Root relaxation: objective 2.559415e+01, 5133 iterations, 0.52 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [9]:
overflow_per_day = (i,t) -> sum(max.(0,
    forecast_start[i] - sum(forecast_start_discharged[i,1:min(t,hospitalized_days)])
    + sum(forecast_admitted[i,max(1,t-hospitalized_days):t])
    - sum(sent[i,:,1:t-1])
    + sum(sent[:,i,max(1,t-hospitalized_days):t])
    - beds[i])
)
overflow = i -> sum(overflow_per_day(i,t) for t=1:size(sent,3));

In [10]:
total_patients = (i,t) -> forecast_start[i] - sum(forecast_start_discharged[i,1:min(t,hospitalized_days)]) + sum(forecast_admitted[i,max(1,t-hospitalized_days):t]) - sum(sent[i,:,1:t]) + sum(sent[:,i,max(1,t-hospitalized_days):t]);

In [11]:
println("Total overflow: ", sum(overflow.(1:length(ne_states))))

Total overflow: 6.86645526002394e-5


In [12]:
summary = DataFrame(
    state=ne_states,
    total_sent=sum(sent, dims=[2,3])[:],
    total_received=sum(sent, dims=[1,3])[:],
    overflow=overflow.(1:length(ne_states)),
)

,state,total_sent,total_received,overflow
,String,Float64,Float64,Float64
1,CT,8030.78,0.0,0.0
2,DE,0.0,976.881,0.0
3,MA,4310.39,1010.01,0.0
4,MD,0.0,1369.22,0.0
5,ME,0.0,2688.47,0.0
6,NH,0.0,1358.81,0.0
7,NJ,3394.74,0.0,0.0
8,NY,5834.37,7693.41,1.81899e-12
9,PA,0.0,5664.11,6.10352e-5


In [13]:
sent_matrix = DataFrame(sum(sent, dims=3)[:,:,1])
rename!(sent_matrix, Symbol.(ne_states))
insertcols!(sent_matrix, 1, :state => ne_states)

,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI,VT
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,684.931,0.0,0.0,2688.47,934.567,0.0,2934.8,0.0,0.0,788.009
2,DE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4310.39,0.0,0.0,0.0
4,MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3394.74,0.0,0.0
8,NY,0.0,291.95,1010.01,1369.22,0.0,424.244,0.0,0.0,2269.37,469.582,0.0
9,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
sent_vis_matrix = sum(sent, dims=3)[:,:,1] + diagm(sum(forecast_admitted, dims=2)[:] - sum(sent, dims=[2,3])[:])
sent_vis_matrix = DataFrame(sent_vis_matrix)
rename!(sent_vis_matrix, Symbol.(ne_states));
# CSV.write("patient_sent_matrix.csv", sent_vis_matrix);

In [15]:
n_days = size(sent,3)
outcomes = DataFrame()
for (i,s) in enumerate(ne_states)
    single_state_outcome = DataFrame(
        state=fill(s, n_days),
        day=start_date .+ Dates.Day.(0:n_days-1),
        sent=sum(sent[i,:,:], dims=1)[:],
        received=sum(sent[:,i,:], dims=1)[:],
        new_patients=forecast_admitted[i,:],
        total_patients=[total_patients(i,t) for t in 1:n_days],
        capacity=fill(beds[i], n_days),
        overflow=[overflow_per_day(i,t) for t in 1:n_days],
        sent_to=[sum(sent[i,:,t])>0 ? collect(zip(ne_states[sent[i,:,t] .> 0], sent[i,sent[i,:,t].>0,t])) : "[]" for t in 1:n_days],
        sent_from=[sum(sent[:,i,t])>0 ? collect(zip(ne_states[sent[:,i,t] .> 0], sent[sent[:,i,t].>0,i,t])) : "[]" for t in 1:n_days],
    )
    outcomes = vcat(outcomes, single_state_outcome)
end
# CSV.write("patient_allocation_results.csv", outcomes)
println("First day:")
filter(row -> row.day == start_date, outcomes)

First day:


,state,day,sent,received,new_patients,total_patients,capacity,overflow,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any,Any
1,CT,2020-04-01,81.259,0.0,81.259,734.991,2159.75,0.0,"[(""ME"", 81.259)]",[]
2,DE,2020-04-01,0.0,36.4938,5.51812,87.943,572.5,0.0,[],"[(""NY"", 36.4938)]"
3,MA,2020-04-01,0.0,144.287,88.4363,765.829,3790.75,0.0,[],"[(""NY"", 144.287)]"
4,MD,2020-04-01,0.0,152.135,180.725,723.555,2793.5,0.0,[],"[(""NY"", 152.135)]"
5,ME,2020-04-01,0.0,81.259,0.0,118.793,887.5,0.0,[],"[(""CT"", 81.259)]"
6,NH,2020-04-01,0.0,53.0305,0.0,81.885,755.0,0.0,[],"[(""NY"", 53.0305)]"
7,NJ,2020-04-01,377.193,0.0,416.61,3526.71,6239.5,0.0,"[(""PA"", 377.193)]",[]
8,NY,2020-04-01,709.241,0.0,1656.37,9862.98,13039.5,0.0,"[(""DE"", 36.4938), (""MA"", 144.287), (""MD"", 152.135), (""NH"", 53.0305), (""PA"", 264.597), (""RI"", 58.6977)]",[]
9,PA,2020-04-01,0.0,641.789,192.774,1534.56,9372.25,0.0,[],"[(""NJ"", 377.193), (""NY"", 264.597)]"


In [16]:
s = "NJ"
filter(row -> row.state == s, outcomes)

,state,day,sent,received,new_patients,total_patients,capacity,overflow,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any,Any
1,NJ,2020-04-01,377.193,0.0,416.61,3526.71,6239.5,0.0,"[(""PA"", 377.193)]",[]
2,NJ,2020-04-02,377.193,0.0,1111.0,3926.52,6239.5,0.0,"[(""PA"", 377.193)]",[]
3,NJ,2020-04-03,377.193,0.0,1321.07,4443.12,6239.5,0.0,"[(""PA"", 377.193)]",[]
4,NJ,2020-04-04,377.193,0.0,993.422,4177.48,6239.5,0.0,"[(""PA"", 377.193)]",[]
5,NJ,2020-04-05,377.193,0.0,1079.6,4350.75,6239.5,0.0,"[(""PA"", 377.193)]",[]
6,NJ,2020-04-06,377.193,0.0,1218.75,4222.16,6239.5,0.0,"[(""PA"", 377.193)]",[]
7,NJ,2020-04-07,377.193,0.0,809.005,4310.53,6239.5,0.0,"[(""PA"", 377.193)]",[]
8,NJ,2020-04-08,377.193,0.0,1277.43,4794.16,6239.5,0.0,"[(""PA"", 377.193)]",[]
9,NJ,2020-04-09,377.193,0.0,1147.62,5564.58,6239.5,0.0,"[(""PA"", 377.193)]",[]


In [17]:
println("Connectivity:")
Dict(ne_states[i] => ne_states[row] for (i,row) in enumerate(eachrow(adj)))

Connectivity:


Dict{String,Array{String,1}} with 11 entries:
  "RI" => ["CT", "DE", "MA", "ME", "NH", "NJ", "NY", "VT"]
  "NJ" => ["CT", "DE", "MA", "MD", "NY", "PA", "RI"]
  "DE" => ["CT", "MD", "NJ", "NY", "PA", "RI"]
  "MD" => ["DE", "NJ", "NY", "PA"]
  "NH" => ["CT", "MA", "ME", "NY", "RI"]
  "NY" => ["CT", "DE", "MA", "MD", "NH", "NJ", "PA", "RI", "VT"]
  "ME" => ["CT", "MA", "NH", "RI", "VT"]
  "CT" => ["DE", "MA", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "MA" => ["CT", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "PA" => ["DE", "MD", "NJ", "NY"]
  "VT" => ["CT", "MA", "ME", "NY", "RI"]

In [18]:
println("Sent to:")
Dict(ne_states[i] => ne_states[row] for (i,row) in enumerate(eachrow(sum(sent, dims=3)[:,:,1] .> 0)))

Sent to:


Dict{String,Array{String,1}} with 11 entries:
  "RI" => ["NY"]
  "NJ" => ["PA"]
  "DE" => String[]
  "MD" => String[]
  "NH" => String[]
  "NY" => ["DE", "MA", "MD", "NH", "PA", "RI"]
  "ME" => String[]
  "CT" => ["DE", "ME", "NH", "NY", "VT"]
  "MA" => ["NY"]
  "PA" => String[]
  "VT" => String[]